In [53]:
import shapely.wkt
import os
from osgeo import ogr

# GFT_REFRESH_TOKEN = os.environ['GFT_REFRESH_TOKEN']

from osgeo import gdal

ogr.UseExceptions()

# gdal.SetConfigOption('CPL_DEBUG', 'ON')
gdal.SetConfigOption('CPL_DEBUG', 'OFF')

use gdal_auth.py to get token, see http://www.gdal.org/drv_gft.html

In [54]:

GFT_REFRESH_TOKEN="1/KWWn5sYPDJGn9NmLeU95mxXWX17Q-E4eL4R_dlq_GH4"


In [1]:
def convert2ft(input_path, output_path, append=False, fix_geometry=False, simplify_geometry=False, start_index=0):
    filename = input_path

    src_ds = ogr.Open(input_path)
    src_lyr = src_ds.GetLayerByIndex(0)
    f = src_lyr.GetFeature(1)

    # create feature table
    dst_ds = ogr.GetDriverByName('GFT').Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
    
    if append:
        dst_lyr = dst_ds.GetLayerByName(output_path)
    else:
        dst_lyr = dst_ds.CreateLayer(output_path)

        # create fields using OGR
        [dst_lyr.CreateField(f.GetFieldDefnRef(i)) for i in range(f.GetFieldCount())]

    index = 0
    batch_size = 15
    index_batch = 0
    for feat in src_lyr:
        if index < start_index:
            index = index + 1
            continue
        
        try:
            geom = shapely.wkt.loads(feat.GetGeometryRef().ExportToWkt())
        except Exception as e: 
            print('Error({0}), skipping geometry.'.format(e))
            continue

        if fix_geometry and not geom.is_valid:
            geom = geom.buffer(0.0)
        
        if simplify_geometry:
            geom = geom.simplify(0.004)

        f = ogr.Feature(dst_lyr.GetLayerDefn())
        
        # set field values
        for i in range(feat.GetFieldCount()):
            fd = feat.GetFieldDefnRef(i)
            f.SetField(fd.GetName(), feat.GetField(fd.GetName()))
            
            #print(fd.GetName())
            #print(feat.GetField(fd.GetName()))
        
        # set geometry    
        f.SetGeometry(ogr.CreateGeometryFromWkt(geom.to_wkt()))
        
        if index_batch == 0:
            dst_lyr.StartTransaction()
            
        index_batch = index_batch + 1
        
        # create feature
        feature = dst_lyr.CreateFeature(f)
        
        f.Destroy() 
        
        index = index + 1

        if index_batch > batch_size:
            dst_lyr.CommitTransaction()
            index_batch = 0
        
    src_ds.Destroy()

    dst_ds.Destroy()

In [49]:
input_file = 'daressalaam-latest-lines.kml'
output_table = 'daressalaam-latest-lines-29-03-2016'


In [ ]:
append = False
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')
    
convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
# convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, 10617) # append

In [7]:
##### append = True
fix_geometry = True
simplify_geometry = False

# gdal.SetConfigOption('CPL_DEBUG', 'OFF')

ds = ogr.Open(input_file)
l = ds.GetLayerByIndex(0)
totalCount = l.GetFeatureCount()

def GetFeatureCount():
    driver = ogr.GetDriverByName('GFT')
    dataSource = driver.Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
    layer = dataSource.GetLayer(12)

    return layer.GetFeatureCount()

count = GetFeatureCount()

print('{0} ({1:.2f}%)'.format(count, 100. * float(count) / totalCount))

while count < totalCount:
    try:
        # convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry) 
        convert2ft(input_file, output_table, append, fix_geometry, simplify_geometry, count) # append
    except RuntimeError:
        count = GetFeatureCount()
        print('{0} ({1:.2f}%)'.format(count, 100. * float(count) / totalCount))

103968 (101.38%)


In [50]:
ds = ogr.Open("daressalaam-latest-lines.kml")
l = ds.GetLayerByIndex(0)
f = l.GetFeature(0)


In [55]:
driver = ogr.GetDriverByName('GFT')
dataSource = driver.Open('GFT:refresh=' + GFT_REFRESH_TOKEN, True)
print(dataSource)


layerCount = dataSource.GetLayerCount()

print('Layer count: {0}'.format(layerCount))

for i in range(layerCount):
    layer = dataSource.GetLayer(i)
    print('{0}: {1}'.format(i, layer.GetName()))
    
    

<osgeo.ogr.DataSource; proxy of <Swig Object of type 'OGRDataSourceShadow *' at 0x7f996f2ab3f0> >
Layer count: 76
0: Catchment_cutoff
1: bounds1
2: daressalaam-latest-polys-17-02-2015
3: Project_Area
4: Merge of daressalaam_polys_cut_attrs and daressalaam_polys_cut
5: daressalaam_polys_cut_attrs
6: daressalaam_polys_cut
7: Merge of daressalaam_lines_attrs and daressalaam_lines
8: daressalaam_lines_attrs
9: daressalaam_polygons
10: daressalaam_lines
11: WB approved admin1
12: SurfaceHydrologyAustralia_Major_MurrayDarling
13: all.shp
14: daressalaam-latest-lines-22-02-2016
15: daressalaam-latest-lines-17-02-2015
16: hybas_as_PFAF44_v1c
17: grid_analysis_Mekong
18: hybas_ar_max4_v1c
19: grid_tiles_Mekong
20: hybas_na_max4_v1c
21: hybas_si_max4_v1c
22: hybas_eu_max4_v1c
23: max4_2
24: max4_1
25: hybas_lev05_v1c
26: hybas_sa_max4_v1c
27: hybas_as_max4_v1c
28: hybas_au_max4_v1c
29: hybas_af_max4_v1c
30: riv_up16000
31: reservoirs_Andijan
32: hybas_lev04_v1c
33: characters_Courier-New
34: cha

In [56]:
dataSource.

RuntimeError: HTML error page returned by server